<a href="https://colab.research.google.com/github/Silent-Zebra/twisted-smc-lm/blob/master/Infillling_Qualitative_Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
from do_training_and_log_Z_bounds import *
import jax
from flax.training import checkpoints
from transformers import AutoTokenizer, FlaxAutoModelForSequenceClassification
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
output_len = 15
n_vocab = 50257
num_last_tokens_to_condition_on = 10

experiment_cfg, rng_key, huggingface_model, params_p, \
params_twist, optimizer_twist, optim_twist_state, \
jnp_prompts, log_true_final_twists, \
true_posterior_samples_by_prompt_and_by_token, records_list_by_prompt_then_twist, \
indices_of_continuation, tokenizer, params_proposal = setup_cfg(
    n_vocab=n_vocab, twist_learn_type="ebm_one_sample", rm_type="p_last_tokens", seed=1,
    hface_model_type="TinyStories", lr_twist=0.0001, beta1=0.9, beta2=0.999, weight_decay=0.0, n_layers_twist=3,
    output_len=output_len, n_samples_at_a_time=0,
    beta_temp=1, threshold=0, pos_threshold=False, load_ckpt=False, load_dirs=(".", "."),
    load_prefix="", hface_nn_twist=True, separate_hface_twist_model=True,
    num_last_tokens_to_condition_on=num_last_tokens_to_condition_on, only_collect_true_posterior_samples=False,
    num_samples_if_only_collect_true_posterior_samples=0, load_posterior_samples=False,
    load_prefix_posterior_samples=None, sentiment_class=5, use_lora=False, softmax_twist=False
)

prompt = jnp_prompts[0]
log_true_final_twist = log_true_final_twists[0]
n_buffer_samples_at_a_time = 0
proposal_is_p = False
tempered_twist=False
beta_prop=None
n_twist = 64

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Using NN for huggingface model twist head


Some weights of the model checkpoint at roneneldan/TinyStories-33M were not used when initializing FlaxGPTNeoForCausalLM: {('transformer', 'h', '2', 'attn', 'attention', 'bias'), ('transformer', 'h', '0', 'attn', 'attention', 'bias'), ('transformer', 'h', '1', 'attn', 'attention', 'masked_bias'), ('transformer', 'h', '2', 'attn', 'attention', 'masked_bias'), ('transformer', 'h', '3', 'attn', 'attention', 'masked_bias'), ('transformer', 'h', '0', 'attn', 'attention', 'masked_bias'), ('transformer', 'h', '1', 'attn', 'attention', 'bias'), ('transformer', 'h', '3', 'attn', 'attention', 'bias')}
- This IS expected if you are initializing FlaxGPTNeoForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing FlaxGPTNeoForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertF

Starting building final twists and getting posterior samples
TIME: 1713411272.3146224
NUM samples
0
[]
(0, 23)
[]
[]
[]
Finished building final twists and getting posterior samples
TIME: 1713411273.2956648


In [ ]:
def print_unique_text(smc_samples, tokenizer):
    text_outputs = tokenizer.batch_decode(smc_samples, skip_special_tokens=True)
    for x in set(text_outputs):
        print(x)

def print_all_text(smc_samples, tokenizer):
    text_outputs = tokenizer.batch_decode(smc_samples, skip_special_tokens=True)
    for x in text_outputs:
        print(x)

def print_all_text_in_order_of_weights(smc_samples, tokenizer, log_w_t):
    print("All Proposal Samples with SMC weights (decreasing order):")
    normalized_log_w_t = jax.nn.softmax(log_w_t)
    text_outputs = tokenizer.batch_decode(smc_samples, skip_special_tokens=True)
    weights_and_outputs = list(reversed(sorted(zip(normalized_log_w_t, text_outputs))))

    for i in range(len(weights_and_outputs)):
        print(f"Weight: {weights_and_outputs[i][0]:.5f} | Text: {weights_and_outputs[i][1]}")


def print_highest_prob_smc_samples(log_w_t, smc_samples):
    text_outputs = tokenizer.batch_decode(smc_samples[log_w_t == max(log_w_t)], skip_special_tokens=True)
    print("Highest weight SMC Sample:")
    for x in set(text_outputs):
        print(x)
    print(f"Weight of highest weight SMC Sample:{max(log_w_t):}")


def do_infilling(rng_key, params_twist, user_specified_condition_text, resample=False):
    condition_text_and_blanks = [user_specified_condition_text, "          "]
    print(f"Prompt: Once upon a time, there was a")
    input_ids_and_mask = tokenizer(condition_text_and_blanks, return_tensors="np", padding=True)  # What happens if the prompts are different lengths? TODO
    jnp_condition_tokens = input_ids_and_mask['input_ids'][0][:num_last_tokens_to_condition_on]
    cut_condition_tokens = tokenizer.batch_decode(jnp_condition_tokens, skip_special_tokens=True)
    print(f"Conditioning tokens (truncated or blank-filled to be 10 tokens): {cut_condition_tokens}")

    condition_twist_on_tokens_broadcasted = jnp.full(
                            (n_twist, jnp_condition_tokens.shape[-1]), jnp_condition_tokens
                        )

    (log_w_t, log_z_hat_t, _), smc_samples = smc_procedure(
                            rng_key, prompt, params_p, params_twist,
                            log_true_final_twist, output_len, n_twist,
                            smc_procedure_type="jit",
                            condition_twist_on_tokens=condition_twist_on_tokens_broadcasted,
                            resample=resample,
                            proposal_is_p=proposal_is_p,
                            huggingface_model=huggingface_model,
                        )

    print_all_text_in_order_of_weights(smc_samples, tokenizer, log_w_t)


def load_ckpt_get_params_twist(ckpt_dir):
    x = checkpoints.restore_checkpoint(ckpt_dir=ckpt_dir, target=None)
    loaded_params_twist = [x['0']['0'], x['0']['1']]

    if 'linear_layers' in loaded_params_twist[1]:
        loaded_params_twist[1]['linear_layers'] = list(loaded_params_twist[1]['linear_layers'].values())
    params_twist = loaded_params_twist
    return params_twist

In [ ]:
params_twist = load_ckpt_get_params_twist("/gdrive/MyDrive/Infilling Demo/checkpoint_2024-02-01_08-12_seed1_one_total_kl_epoch12")

In [ ]:
do_infilling(jax.random.PRNGKey(0), params_twist, " others and make them feel safe. Mia liked to") # DPG Results

Prompt: Once upon a time, there was a
Conditioning tokens (truncated or blank-filled to be 10 tokens): [' others', ' and', ' make', ' them', ' feel', ' safe', '.', ' Mia', ' liked', ' to']
(64, 23, 1536)
(64, 23, 1536)
All Proposal Samples with SMC weights (decreasing order):
Weight: 0.59397 | Text: Once upon a time, there was a little girl named Mia. She had a big heart. Mia loved to help
Weight: 0.28629 | Text: Once upon a time, there was a girl named Mia. Mia was a very happy girl because she liked to help
Weight: 0.10429 | Text: Once upon a time, there was a little girl named Mia. She had a big job. She was to help
Weight: 0.01391 | Text: Once upon a time, there was a brave girl named Lily. She had a special job - she liked to help
Weight: 0.00073 | Text: Once upon a time, there was a girl named Mia. Mia was a kind girl who liked to lend things to
Weight: 0.00048 | Text: Once upon a time, there was a little girl named Mia. She was a generous girl who liked to share with
Weight: 0.0

In [ ]:
# Can run out of memory on colab. If so, just skip the previous 2 cells before running this
params_twist = load_ckpt_get_params_twist("/gdrive/MyDrive/Infilling Demo/checkpoint_2024-02-01_08-21_seed1_sixo_epoch12")

In [ ]:
do_infilling(jax.random.PRNGKey(0), params_twist, " others and make them feel safe. Mia liked to") # SIXO Results

Prompt: Once upon a time, there was a
Conditioning tokens (truncated or blank-filled to be 10 tokens): [' others', ' and', ' make', ' them', ' feel', ' safe', '.', ' Mia', ' liked', ' to']
(64, 23, 1536)
(64, 23, 1536)
All Proposal Samples with SMC weights (decreasing order):
Weight: 0.69534 | Text: Once upon a time, there was a girl named Mia. Mia was very kind and compassionate. She always helped her
Weight: 0.09194 | Text: Once upon a time, there was a little girl named Mia. She loved playing with dough. She used to make
Weight: 0.05874 | Text: Once upon a time, there was a little girl named Mia. Mia loved her daddy very much. Her daddy was
Weight: 0.05448 | Text: Once upon a time, there was a little girl named Sue. Sue was a very reliable girl. She always helped
Weight: 0.03489 | Text: Once upon a time, there was a little girl named Sue. Sue had a big dream. She wanted to bring
Weight: 0.02419 | Text: Once upon a time, there was a jolly girl named Mia. She loved to play with her fr

In [ ]:
# Can run out of memory on colab. If so, just skip the previous 4 cells before running this
params_twist = load_ckpt_get_params_twist("/gdrive/MyDrive/Infilling Demo/checkpoint_2024-02-01_10-49_seed1_ebm_ml_jit_vmapped_over_condition_tokens_epoch12")

In [ ]:
do_infilling(jax.random.PRNGKey(0), params_twist, " others and make them feel safe. Mia liked to") # CTL Results

Prompt: Once upon a time, there was a
Conditioning tokens (truncated or blank-filled to be 10 tokens): [' others', ' and', ' make', ' them', ' feel', ' safe', '.', ' Mia', ' liked', ' to']
(64, 23, 1536)
(64, 23, 1536)
All Proposal Samples with SMC weights (decreasing order):
Weight: 0.93947 | Text: Once upon a time, there was a girl named Mia. She had a thin, pink dress. Mia liked to
Weight: 0.05986 | Text: Once upon a time, there was a little girl named Amy. She was a happy girl who wanted to go to
Weight: 0.00043 | Text: Once upon a time, there was a girl named Sarah. She was a very happy girl and she loved to share
Weight: 0.00008 | Text: Once upon a time, there was a little girl named Sally. Sally was very spoiled and she loved to ask questions
Weight: 0.00004 | Text: Once upon a time, there was a little girl named Sarah. Sarah was 3 years old and loved to play with
Weight: 0.00003 | Text: Once upon a time, there was a little girl called Sally. She loved playing with her toys and 